# 实例-搭建疫情数据分析网站

## 0 实例介绍

## 1 获取数据

## 2 网站`html`文件编辑

## 3 基础图像的绘制

本项目中，我们仍然使用`covid19`数据集，该数据集收集了31个身份53天内的疫情数据，我们参考JHU疫情分析以及百度大数据疫情分析网站，来制作基础的图像。

In [22]:
import pandas as pd 
import numpy as np 
from bokeh.plotting import figure, output_file, show
from bokeh.plotting import ColumnDataSource as CDS 
from bokeh.models import Toggle
from bokeh.layouts import layout, column, row

In [48]:
covid19 = pd.read_csv('covid19.csv')
covid19['Observation Date'] = pd.to_datetime(covid19['Observation Date'], format = '%d-%m-%Y')

In [49]:
provincedata = covid19.groupby('Province/State')[['Confirmed', 'Observation Date']]

In [50]:
cityname = []
confirmed = []
deaths = []
latitude = []
longitude = []
date = list(provincedata)[0][1]['Observation Date']

for city in list(provincedata):
    cityname.append(city[0])
    confirmed.append(list(city[1]['Confirmed']))
    deaths.append(list(city[1]['Deaths']))
    latitude.append(city[1]['Latitude'].iloc[0])
    longitude.append(city[1]['Longitude'].iloc[0])

Confirmed = pd.DataFrame(pd.DataFrame(confirmed).values.T, index = date, columns = cityname)
Deaths = pd.DataFrame(pd.DataFrame(deaths).values.T, index = date, columns = cityname)

In [86]:
source = CDS(Confirmed)

TOOLTIPS = [
    ('date', '$x'),
    ('Confirmed Infections', '$y'),
    ('Province', '@Province')
]

pa = figure( x_axis_type = 'datetime', tooltips = TOOLTIPS)

hbconfirmed = pd.DataFrame(Confirmed['Hubei'])
hbconfirmed['date'] = hbconfirmed.index
Confirmed2 = Confirmed.drop('Hubei', axis = 1)

finalshow = []
for data in range(30):
    confirmed = pd.DataFrame(Confirmed2.iloc[:, data])
    confirmed['Province'] = np.repeat(confirmed.columns[0], 54)
    confirmed['date'] = confirmed.index
    source = CDS(confirmed)
    p1 = pa.line(y = confirmed.columns[0] , x = 'date',source = source,  line_width = 2)
    pa.add_layout(p1)
    p2 = pa.circle(y = confirmed.columns[0] , x = 'date',source = source, width = 0.6, color = 'navy', alpha = 0.5)
    pa.add_layout(p2)

pb = figure( x_axis_type = 'datetime', tooltips = TOOLTIPS)
source = CDS(hbconfirmed)
pb.line(y = 'Hubei', x = 'date',source = source, line_width = 2)
pb.circle(y = 'Hubei', x = 'date',source = source, width = 0.6, color = 'navy', alpha = 0.5)


show(row(pa,pb))